In [ ]:
#rtWIZARD Dry Run Protocol v1.0.0
#Will dry run protocol v1.0.0 using the same tip

#import opentrons.execute if you want to execute the protocol on the OT-2
#import opentrons.execute
#protocol = opentrons.execute.get_protocol_api("2.13")
#protocol.home()    #homes motors

#import opentrons.simulate if you want to simulate the protocol first
import opentrons.simulate
metadata = {
    "apiLevel": "2.13",
    "schemaVersion": "2"}
protocol = opentrons.simulate.get_protocol_api('2.13')
protocol.home()    #homes motors

#only for simulation/duration timer: get file path
#import os
#notebook_path = os.getcwd()
#print(notebook_path)

#assigns each OT2 slot a custom labware definition that corresponds to the cutout
#BEFORE EXECUTING: Run labware position check and copy/paste the Jupyter Notebook labware offsets as shown below:
'''
slot1 = protocol.load_labware("rtwizard_slots_147", 1)
slot1.set_offset(x=1.30, y=-3.50, z=-4.00)
'''
#paste offsets below (do not include opentrons.execute):

labware_1 = protocol.load_labware("rtwizard_slot_147_rapid_asfv_ag", location="1")
labware_1.set_offset(x=-6.50, y=-4.30, z=-5.00)

labware_2 = protocol.load_labware("rtwizard_slot_258_rapid_asfv_ag", location="2")
labware_2.set_offset(x=-1.20, y=-4.40, z=-5.00)

labware_3 = protocol.load_labware("rtwizard_slot_369_rapid_asfv_ag", location="3")
labware_3.set_offset(x=4.00, y=-4.00, z=-5.00)

labware_4 = protocol.load_labware("rtwizard_slot_147_rapid_asfv_ag", location="4")
labware_4.set_offset(x=-6.50, y=-2.00, z=-4.60)

labware_5 = protocol.load_labware("rtwizard_slot_258_rapid_asfv_ag", location="5")
labware_5.set_offset(x=-1.00, y=-2.00, z=-4.00)

labware_6 = protocol.load_labware("rtwizard_slot_369_rapid_asfv_ag", location="6")
labware_6.set_offset(x=3.80, y=-2.00, z=-4.00)

labware_7 = protocol.load_labware("rtwizard_slot_147_rapid_asfv_ag", location="7")
labware_7.set_offset(x=-6.10, y=-0.30, z=-4.00)

labware_8 = protocol.load_labware("rtwizard_slot_258_rapid_asfv_ag", location="8")
labware_8.set_offset(x=-0.40, y=0.50, z=-4.00)

labware_9 = protocol.load_labware("rtwizard_slot_369_rapid_asfv_ag", location="9")
labware_9.set_offset(x=4.90, y=0.20, z=-4.00)

labware_10 = protocol.load_labware("rtwizard_48_wellplate_sarstedt_1100ul_tubes", location="10")
labware_10.set_offset(x=-1.00, y=2.00, z=-3.00)

labware_11 = protocol.load_labware("opentrons_96_tiprack_300ul", location="11")
labware_11.set_offset(x=-0.60, y=7.40, z=0.00)

#standardize names
sample_plate = labware_10
tips = labware_11

#comment or uncomment depending on pipette setup
p300 = protocol.load_instrument('p300_single_gen2', 'left', tip_racks=[tips])
#p20 = protocol.load_instrument('p20_single_gen2', 'right', tip_racks=[tips])

def main():

    #test-defined values
    #Adjust these values based on the lateral flow assay protocol
    
    #INGEZIM ASF CROM Ag
    num_wells = 1
    duo_config = False
    uses_buffer = True
    immediately_add_buffer = False
    leftmost_well_for_sample = True
    buffer_in_which_well = 1
    tubes_1100 = True
    tubes_2000 = False
    
    num_drops_sample = 1
    sample_vol = 50 #ul
    sample_vol_per_drop = 50

    buffer_vol = 150  
    
    #for simulation only
    protocol.home()
    
    #DRY RUN: pick up tip
    p300.pick_up_tip(tips.wells()[0])
    
    #transfer sample liquid to cassettes
    #transfer_samples(sample_vol)
    
    #transfer X drops of running buffer to cassettes
    if(uses_buffer == True):
        transfer_buffer(buffer_vol, tubes_1100, tubes_2000)
    
    #DRY RUN: drop tip
    p300.drop_tip(tips.wells()[0])
    
    #rehome
    protocol.home()
    
    #print console output
    for line in protocol.commands():
        print(line)

#fin
        
def transfer_samples(sample_vol):
    for i in range(5):

        #determining which slots to dispense on for this iteration of the loop
        if (i == 0 or i == 1):
            start_slot = labware_1
            mid_slot = labware_4
            fin_slot = labware_7
        elif (i == 2):
            start_slot = labware_2
            mid_slot = labware_5
            fin_slot = labware_8
        else:
            start_slot = labware_3
            mid_slot = labware_6
            fin_slot = labware_9

        #loading_wells: list of wells being loaded with sample liquid
        if ((i % 2 == 0 and i != 4) or i == 3):
            loading_wells = ['C1', 'B1', 'A1']
            loading_wells2 = ['C1', 'B1']
        else:
            loading_wells = ['C2', 'B2', 'A2']
            loading_wells2 = ['C2', 'B2']
        
        #transfers
        row = sample_plate.rows()[i]
        p300.transfer(
            sample_vol, 
            row[:3], 
            [start_slot.wells_by_name()[well_name] for well_name in loading_wells],
            new_tip='never')   #trash=False will cause pipette with tip to hover over next tip
        p300.transfer(
            sample_vol, 
            row[3:6], 
            [mid_slot.wells_by_name()[well_name] for well_name in loading_wells],
            new_tip='never')
        p300.transfer(
            sample_vol, 
            row[6:8], 
            [fin_slot.wells_by_name()[well_name] for well_name in loading_wells2],
            new_tip='never')

    p300.transfer(
        sample_vol, 
        [sample_plate.wells_by_name()[well_name] for well_name in ['F1', 'F2']], 
        [labware_7.wells_by_name()[well_name] for well_name in ['A1', 'A2']],
        new_tip='never')

def transfer_buffer(buffer_vol, tubes_1100, tubes_2000):
    
    #listing wells F3-F8 which contain running buffer
    buffer_wells = ['F3', 'F4', 'F5', 'F6', 'F7', 'F8']
    #dict that determines which destination wells should use which source well for buffer
    wells_per_tube = {}
    
    #define wells_per_tube
    if tubes_1100 == True:
        num_wells_per_tube = int(1100/buffer_vol)
        which_wells_per_tube = [num_wells_per_tube * i for i in range(1, 7)]
        wells_per_tube = dict(zip(which_wells_per_tube, buffer_wells))
        print(wells_per_tube)
    
    #determines when to switch to new buffer tube
    well_count = 0
    #number of the current buffer tube being drawn from, starting from 0
    curr_tube = 0
    switch_tubes = False
    
    #distribute buffer to wells
    for i in range(5):
        
        #determining which slots to dispense on for this iteration of the loop
        if (i == 0 or i == 1):
            start_slot = labware_1
            mid_slot = labware_4
            fin_slot = labware_7
        elif (i == 2):
            start_slot = labware_2
            mid_slot = labware_5
            fin_slot = labware_8
        else:
            start_slot = labware_3
            mid_slot = labware_6
            fin_slot = labware_9

        #loading_wells: list of wells being loaded with running buffer
        if ((i % 2 == 0 and i != 4) or i == 3):
            loading_wells = ['C1', 'B1', 'A1']
            loading_wells2 = ['C1', 'B1']
        else:
            loading_wells = ['C2', 'B2', 'A2']
            loading_wells2 = ['C2', 'B2']
        
        if well_count >= wells_per_tube[curr_tube]:
            switch_tubes = True
            curr_tube += 1
        
        #individual transfers for RB
        for j in range(8):
            p300.transfer(
                sample_vol, 
                row[:3], 
                [start_slot.wells_by_name()[well_name] for well_name in loading_wells],
                new_tip='never')
        
        #distributes
        #TODO: determine if disposal_volume should be increased for more consistent dispense volumes at the cost of time and buffer
        p300.distribute(
            buffer_vol, 
            [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
            [start_slot.wells_by_name()[well_name] for well_name in loading_wells], 
            new_tip='never',
            disposal_volume=0)
        p300.distribute(
            buffer_vol, 
            [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
            [mid_slot.wells_by_name()[well_name] for well_name in loading_wells],
            new_tip='never',
            disposal_volume=0)
        p300.distribute(
            buffer_vol, 
            [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
            [fin_slot.wells_by_name()[well_name] for well_name in loading_wells2],
            new_tip='never',
            disposal_volume=0)

    p300.distribute(
        buffer_vol, 
        [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
        [labware_7.wells_by_name()[well_name] for well_name in ['A1', 'A2']],
        new_tip='never',
        disposal_volume=0)
    
if __name__ == "__main__":
    main()